<center>
    <h1>C++ kernel based on xeus</h1>
</center>

A Jupyter kernel for C++ based on the `CppInterOp`, a clang based C++ Interoperability Library and the `xeus` native implementation of the Jupyter protocol, xeus.

- GitHub repository: https://github.com/compiler-research/xeus-cpp
- Documentation: https://xeus-cpp.readthedocs.io/en/latest/

## Usage

<div style="background: #efffed;
            border: 1px solid grey;
            margin: 8px 0 8px 0;
            text-align: center;
            padding: 8px; ">
    <i class="fa-play fa" 
       style="font-size: 40px;
              line-height: 40px;
              margin: 8px;
              color: #444;">
    </i>
    <div>
    To run the selected code cell, hit <pre style="background: #efffed">Shift + Enter</pre>
    </div>
</div>

## Output and error streams

`std::cout` and `std::cerr` are redirected to the notebook frontend.

In [1]:
#include <iostream>

std::cout << "some output" << std::endl;

some output


In [2]:
std::cerr << "some error" << std::endl;

some error


In [3]:
#include <stdexcept>

try {
    throw std::runtime_error("Unknown exception");
}
catch (const std::exception& e) {
    std::cout << "Caught an exception: " << e.what() << std::endl;
}

Caught an exception: Unknown exception


In [4]:
int j = 5;
std::cout << j << std::endl;

5


# Interpreting the C++ programming language
# You can define functions, classes, templates, etc ...

## Functions

In [5]:
double sqr(double a)
{
    return a * a;
}

In [6]:
double a = 2.5;
double asqr = sqr(a);
std::cout << asqr << std::endl;

6.25


## Classes

In [7]:
class Foo
{
public:

    virtual ~Foo() {}

    virtual void print(double value) const
    {
        std::cout << "Foo value = " << value << std::endl;
    }
};

In [8]:
Foo bar;
bar.print(1.2);

Foo value = 1.2


## Polymorphism

In [9]:
class Bar : public Foo
{
public:

    virtual ~Bar() {}

    virtual void print(double value) const
    {
        std::cout << "Bar value = " << 2 * value << std::endl;
    }
};

In [10]:
Foo* bar2 = new Bar;
bar2->print(1.2);
delete bar2;

Bar value = 2.4


## Templates

In [11]:
#include <typeinfo>

template <class T>
class FooT
{
public:

    explicit FooT(const T& t) : m_t(t) {}

    void print() const
    {
        std::cout << typeid(T).name() << " m_t = " << m_t << std::endl;
    }

private:

    T m_t;
};

In [12]:
FooT<double> foot(1.2);
foot.print();

d m_t = 1.2


## Documentation

 - Documentation for types of the standard library is retrieved on cppreference.com.

In [13]:
?std::vector

https://en.cppreference.com/w/cpp/container/vector

## Using the display_data mechanism

For a user-defined type `T`, the rich rendering in the notebook and JupyterLab can be enabled by by implementing the function `nl::json mime_bundle_repr(const T& im)`, which returns the JSON mime bundle for that type.

### Image example

In [14]:
#include <string>
#include <fstream>
#include <sstream>

#include "nlohmann/json.hpp"

#include "xeus/xbase64.hpp"

namespace nl = nlohmann;

namespace im
{
    struct image
    {
        inline image(const std::string& filename)
        {
            std::ifstream fin(filename, std::ios::binary);
            m_buffer << fin.rdbuf();
        }

        std::stringstream m_buffer;
    };

    nl::json mime_bundle_repr(const image& i)
    {
        auto bundle = nl::json::object();
        bundle["image/png"] = xeus::base64encode(i.m_buffer.str());
        return bundle;
    }
}

In [15]:
im::image marie("../data/marie.png");

In [16]:
#include "xcpp/xdisplay.hpp"

In [17]:
xcpp::display(marie);

### Audio example

In [18]:
namespace nl = nlohmann;

namespace au
{
    struct audio
    {
        inline audio(const std::string& filename)
        {
            std::ifstream fin(filename, std::ios::binary);
            m_buffer << fin.rdbuf();
        }

        std::stringstream m_buffer;
    };

    nl::json mime_bundle_repr(const audio& a)
    {
        auto bundle = nl::json::object();
        bundle["text/html"] =
           std::string("<audio controls=\"controls\"><source src=\"data:audio/wav;base64,")
           + xeus::base64encode(a.m_buffer.str()) +
            "\" type=\"audio/wav\" /></audio>";
        return bundle;
    }
}

In [19]:
au::audio drums("../data/audio.wav");

In [20]:
xcpp::display(drums);

### Update-display

In [21]:
namespace nl = nlohmann;

namespace ht
{
    struct html
    {
        inline html(const std::string& content)
        {
            m_content = content;
        }
        std::string m_content;
    };

    nl::json mime_bundle_repr(const html& a)
    {
        auto bundle = nl::json::object();
        bundle["text/html"] = a.m_content;
        return bundle;
    }
}

// A blue rectangle
ht::html rect(R"(
<div style='
    width: 90px;
    height: 50px;
    line-height: 50px;
    background-color: blue;
    color: white;
    text-align: center;'>
Original
</div>)");

In [22]:
xcpp::display(rect, "some_display_id");

Updated

In [23]:
// Update the rectangle to be red
rect.m_content = R"(
<div style='
    width: 90px;
    height: 50px;
    line-height: 50px;
    background-color: red;
    color: white;
    text-align: center;'>
Updated
</div>)";

xcpp::display(rect, "some_display_id", true);

### Clear output

In [24]:
#include <chrono>
#include <thread>

In [25]:
std::cout << "hello" << std::endl;
std::this_thread::sleep_for(std::chrono::seconds(1));
xcpp::clear_output();  // will flicker when replacing "hello" with "goodbye"
std::this_thread::sleep_for(std::chrono::seconds(1));
std::cout << "goodbye" << std::endl;

goodbye


In [26]:
std::cout << "hello" << std::endl;
std::this_thread::sleep_for(std::chrono::seconds(1));
xcpp::clear_output(true);  // prevents flickering
std::this_thread::sleep_for(std::chrono::seconds(1));
std::cout << "goodbye" << std::endl;

goodbye


### SIMD Acceleration through WebAssembly Intrinsics

Example inspired by https://emscripten.org/docs/porting/simd.html#webassembly-simd-intrinsics

In [27]:
#include <wasm_simd128.h>
#include <stdio.h>

In [28]:
v128_t v1 = wasm_f32x4_make(1.2f, 3.4f, 5.6f, 7.8f);
v128_t v2 = wasm_f32x4_make(2.1f, 4.3f, 6.5f, 8.7f);

v128_t v3 = wasm_f32x4_add(v1, v2);

printf("v3: [%.1f, %.1f, %.1f, %.1f]\n",
    wasm_f32x4_extract_lane(v3, 0),
    wasm_f32x4_extract_lane(v3, 1),
    wasm_f32x4_extract_lane(v3, 2),
    wasm_f32x4_extract_lane(v3, 3));

v3: [3.3, 7.7, 12.1, 16.5]
